In [ ]:
%load_ext autoreload
%autoreload 2
from dolfin import *
from multiphenics import *
import matplotlib.pyplot as plt
from dolfin import plot
from braininversion.BiotNavierStokesSolver import solve_biot_navier_stokes

outfile = "../results/membrane_flow.xdmf"
T = 20
num_steps = 20

n = 20
ratio = 4.0
interf = 2.0

interf_id = 1
top_id = 2
bottom_id = 3
right_id = 4
left_id = 5

fluid_id = 2
porous_id = 1
mesh = RectangleMesh(Point(0.0), Point(ratio, 1.0), int(n*ratio), n)

fluid = CompiledSubDomain("x[0] <= interf + DOLFIN_EPS", interf=interf)
porous = CompiledSubDomain("x[0] >= interf - DOLFIN_EPS", interf=interf)

bottom = CompiledSubDomain("on_boundary && near(x[1], 0.0)")
top = CompiledSubDomain("on_boundary && near(x[1], 1.0)")
left = CompiledSubDomain("on_boundary && near(x[0], 0.0)")
right = CompiledSubDomain("on_boundary && near(x[0], end)", end=ratio)
interface = CompiledSubDomain("near(x[0], interf, DOLFIN_EPS)", interf=interf)


boundaries = MeshFunction("size_t", mesh, mesh.topology().dim() -1, 0)
bottom.mark(boundaries, bottom_id)
top.mark(boundaries, top_id)
right.mark(boundaries, right_id)
left.mark(boundaries, left_id)
interface.mark(boundaries, interf_id)

subdomains = MeshFunction("size_t", mesh, mesh.topology().dim(), 0)
porous.mark(subdomains, porous_id)
fluid.mark(subdomains, fluid_id)

# define constants
A = 0.01
E = 1500               # Young modulus
nu = 0.4799             # Poisson ratio

material_parameter = dict()
material_parameter["rho_s"] = Constant(1e3)
material_parameter["mu_s"] = Constant(E/(2.0*(1.0+nu)))

material_parameter["rho_f"] = Constant(1e3)
material_parameter["mu_f"] = Constant(1e-3)

material_parameter["lmbda"] = Constant(nu*E/((1.0-2.0*nu)*(1.0+nu)))
material_parameter["alpha"] = Constant(1.0)
material_parameter["kappa"] = Constant(1e-4)
material_parameter["c"] = Constant(1e-7)

# define sources and forces

g = Constant([0.0, 0.0])
f = Constant([0.0, 0.0])
g_source = Constant(0.00)

# define boundary conditions (fluid left and porous right)

inflow = Expression(("A*sin(x[1]*pi)", "0.0"), A=A, degree=2)


boundary_conditions = [
    {left_id: {0 : inflow},},
    {top_id : {0: Constant([0.0, 0.0])}},
    {bottom_id : {0: Constant([0.0, 0.0])}},
    {top_id : {2: Constant([0.0, 0.0])}},
    {bottom_id : {2: Constant([0.0, 0.0])}},
    {right_id : {2: Constant([0.0, 0.0])}},
    {right_id :{3: Constant(0.0)}}
    ]

results = solve_biot_navier_stokes(mesh, T, num_steps,
                                   material_parameter, 
                                   boundaries,
                                   subdomains,
                                   boundary_conditions,
                                   g_source=g_source,
                                   filename=outfile)
    


In [ ]:
for i, func in enumerate(results):
    plt.figure(figsize=(15,4))
    c = plot(func)
    plt.title(func.name())
    plt.colorbar(c)

In [ ]:
pP = results[3]

In [ ]:
pP.vector()[:].mean()